In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

class CricketMatchPredictor:
    def __init__(self):
        self.data = {}
        self.models = {}
        self.combined_data = None

    def load_data(self, file_paths):
        """Load multiple stadium Excel files into memory"""
        all_data = []
        for stadium, path in file_paths.items():
            try:
                df = pd.read_excel(path)

                # --- clean ---
                # drop completely empty rows
                df = df.dropna(how="all")

                # if first row looks like headers repeated inside data, drop it
                if str(df.iloc[0,0]).lower() in ["year", "years"] or str(df.iloc[0,1]).lower().startswith("first"):
                    df = df.iloc[1:].reset_index(drop=True)

                # ensure correct column names
                df.columns = ["Year", "First Innings", "Second Innings", "Stadium"]

                # convert scores to numeric
                df["First Innings"] = pd.to_numeric(df["First Innings"], errors="coerce")
                df["Second Innings"] = pd.to_numeric(df["Second Innings"], errors="coerce")
                df = df.dropna(subset=["First Innings", "Second Innings"])

                # derive win column
                df["First_Win"] = (df["First Innings"] > df["Second Innings"]).astype(int)

                self.data[stadium] = df
                all_data.append(df)
                print(f"✅ Loaded {len(df)} matches from {stadium}")

            except Exception as e:
                print(f"❌ Error loading {stadium}: {e}")

        if not all_data:
            raise ValueError("No valid stadium files loaded")

        # combine all for model training
        self.combined_data = pd.concat(all_data, ignore_index=True)
        print(f"📊 Total matches loaded: {len(self.combined_data)}")

    def train_models(self):
        """Train regression and classification models on combined data"""
        if self.combined_data is None:
            raise ValueError("Load data first")

        df = self.combined_data.copy()
        X = df[["First Innings"]]
        y_reg = df["Second Innings"]
        y_clf = df["First_Win"]

        # regression (second innings prediction)
        X_train, X_test, y_train, y_test = train_test_split(X, y_reg, test_size=0.2, random_state=42)
        rf = RandomForestRegressor(random_state=42, n_estimators=200)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
        print(f"🌲 RF MAE: {mean_absolute_error(y_test, y_pred):.2f}, R²: {r2_score(y_test, y_pred):.3f}")

        lr = LinearRegression()
        lr.fit(X_train, y_train)
        y_pred_lr = lr.predict(X_test)
        print(f"📈 LR MAE: {mean_absolute_error(y_test, y_pred_lr):.2f}, R²: {r2_score(y_test, y_pred_lr):.3f}")

        # classification (win/loss)
        clf = LogisticRegression()
        clf.fit(X, y_clf)

        self.models = {"regressor": rf, "classifier": clf}

    def stadium_summary(self, stadium):
        """Compute summary stats for a stadium"""
        if stadium not in self.data:
            return {"error": f"No data for {stadium}"}
        df = self.data[stadium]
        total_matches = len(df)
        avg_first = df["First Innings"].mean()
        avg_second = df["Second Innings"].mean()
        avg_winning_first = df.loc[df["First_Win"] == 1, "First Innings"].mean()
        win_rate = df["First_Win"].mean() * 100
        return {
            "stadium": stadium,
            "total_matches": total_matches,
            "avg_first_innings": round(avg_first,1),
            "avg_second_innings": round(avg_second,1),
            "avg_winning_first_innings": round(avg_winning_first,1),
            "first_win_rate(%)": round(win_rate,1)
        }

    def predict_match(self, stadium, first_innings_score=None):
        """Predict outcome given stadium + optional first innings score"""
        if stadium not in self.data:
            return {"error": f"No data for {stadium}"}

        if first_innings_score is None:
            # yet to bat
            return self.stadium_summary(stadium)

        # predict second innings
        reg = self.models["regressor"]
        clf = self.models["classifier"]
        pred_second = reg.predict([[first_innings_score]])[0]
        win_prob = clf.predict_proba([[first_innings_score]])[0][1]
        margin = first_innings_score - pred_second
        result = "First Innings Wins" if margin > 0 else "Chasing Wins"

        return {
            "stadium": stadium,
            "input_first_innings": first_innings_score,
            "predicted_second_innings": round(pred_second,1),
            "predicted_margin": round(abs(margin),1),
            "predicted_result": result,
            "win_probability_first": round(win_prob*100,1)
        }

# ----------------- USAGE EXAMPLE -------------------

if __name__ == "__main__":
    # define your file paths (update with actual Colab paths)
    file_paths = {
        "Chepauk": "/content/Chepauk.xlsx",
        "Mumbai": "/content/Mumbai.xlsx",
        "Kolkata": "/content/Kolkata.xlsx",
        "Lucknow": "/content/Lucknow.xlsx",
        "Hyderabad": "/content/Hyderabad.xlsx"
    }

    predictor = CricketMatchPredictor()
    predictor.load_data(file_paths)
    predictor.train_models()

    # query examples
    print("\nYet to bat (Chepauk):")
    print(predictor.predict_match("Chepauk", None))

    print("\nFirst Innings = 250 (Chepauk):")
    print(predictor.predict_match("Chepauk", 250))


✅ Loaded 23 matches from Chepauk
✅ Loaded 21 matches from Mumbai
✅ Loaded 20 matches from Kolkata
✅ Loaded 21 matches from Lucknow
✅ Loaded 18 matches from Hyderabad
📊 Total matches loaded: 103
🌲 RF MAE: 23.62, R²: -0.007
📈 LR MAE: 17.94, R²: 0.219

Yet to bat (Chepauk):
{'stadium': 'Chepauk', 'total_matches': 23, 'avg_first_innings': np.float64(167.7), 'avg_second_innings': np.float64(155.9), 'avg_winning_first_innings': np.float64(189.2), 'first_win_rate(%)': np.float64(39.1)}

First Innings = 250 (Chepauk):
{'stadium': 'Chepauk', 'input_first_innings': 250, 'predicted_second_innings': np.float64(247.1), 'predicted_margin': np.float64(2.9), 'predicted_result': 'First Innings Wins', 'win_probability_first': np.float64(83.7)}


In [4]:
# ---------------- Interactive Prediction (Formatted Output) ----------------

stadium = input("Enter stadium name (Chepauk / Mumbai / Kolkata / Lucknow / Hyderabad): ").strip()
score = input("Enter first innings score (or type 'yet' if yet to bat): ").strip()

if score.lower() == "yet":
    result = predictor.predict_match(stadium, None)
else:
    try:
        score_val = int(score)
        result = predictor.predict_match(stadium, score_val)
    except:
        result = {"error": "Invalid score entered"}

print("\n🔮 Prediction Result:")

if "error" in result:
    print("❌", result["error"])
else:
    # Pretty formatting
    for key, value in result.items():
        if isinstance(value, float):
            print(f"{key:25}: {value:.1f}")
        else:
            print(f"{key:25}: {value}")


Enter stadium name (Chepauk / Mumbai / Kolkata / Lucknow / Hyderabad): Chepauk
Enter first innings score (or type 'yet' if yet to bat): yet to bat

🔮 Prediction Result:
❌ Invalid score entered


# Task
Save the trained `predictor.models` object to a file named `cricket_predictor_models.pkl`.

## Save Trained Models

### Subtask:
Save the trained `predictor.models` object to a file named `cricket_predictor_models.pkl` using Python's `pickle` library.


## Summary:

### Data Analysis Key Findings
*   The trained `predictor.models` object was successfully saved to a file named `cricket_predictor_models.pkl` on the local file system.
*   The `pickle` library was used for serialization, making the model object readily available for future loading and use.

### Insights or Next Steps
*   This saved model can now be easily deployed or used for making predictions without retraining, ensuring reproducibility and efficiency.
*   Consider versioning the saved models to track changes and improvements over time, especially when retraining with new data or different parameters.


In [6]:
from google.colab import files
files.download('cricket_predictor_models.pkl')

FileNotFoundError: Cannot find file: cricket_predictor_models.pkl

In [7]:
import pickle

# Save the trained models object to a file
with open('cricket_predictor_models.pkl', 'wb') as f:
    pickle.dump(predictor.models, f)

print("✅ Trained models saved to cricket_predictor_models.pkl")

✅ Trained models saved to cricket_predictor_models.pkl
